In [1]:
# get summaries of story from GPT 4

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [27]:
from pathlib import Path
stories_path = "../dataset/stories"
pathlist = Path(stories_path).rglob('*.txt')

In [28]:
def storyfile_processor(story_file_path):
    with open(story_file_path, encoding="utf8") as f:
        content = f.read()
        title = content.split("===")[0].strip()
        author = content.split("===")[1].strip()
        story_text = content.split("===")[2].strip()
        print(f"Title: {title}, Author: {author}")
    return (title, author, story_text)

In [29]:
def story_processor(story):
    # remove U+2019 character, replace it with apostrophe
    # remove additional spaces
    # remove double quotes that are like repeated U+2019
    pass

In [30]:
stories = []
for path in sorted(pathlist):
    # because path is object not string
    path_in_str = str(path)
    print(path_in_str)
    title, author, story = storyfile_processor(path_in_str)
    stories.append((title, author, story))
stories_df = pd.DataFrame(stories, columns=["Title", "Author", "story"])    

..\dataset\stories\1.txt
Title: The Bear Came Over the Mountain, Author: Alice Munro
..\dataset\stories\10.txt
Title: The Boss, Author: Robert Coover
..\dataset\stories\11.txt
Title: The Hairless Are Careless, Author: Colin Barrett
..\dataset\stories\12.txt
Title: Smithereens, Author: Aleksandar Hemon
..\dataset\stories\13.txt
Title: Where Are You?, Author: Joyce Carol Oates
..\dataset\stories\14.txt
Title: Art of Story, Author: John Edgar Wideman
..\dataset\stories\15.txt
Title: Sun Dogs, Author: Dorthe Nors
..\dataset\stories\16.txt
Title: Let’s Say, Author: Linor Goralik
..\dataset\stories\17.txt
Title: Citizen Punch, Author: Robert Coover
..\dataset\stories\18.txt
Title: An Evening with Joseph Conrad, Author: Anne Carson
..\dataset\stories\19.txt
Title: The South Asian Speakers Series Presents the Archeologist and Adventurer Indiana Jones, Author: Tania James
..\dataset\stories\2.txt
Title: Town of Cats, Author: Haruki Murakami
..\dataset\stories\20.txt
Title: Courage, Author: Dani

In [8]:
stories_df.head(1)

,Title,Author,story
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town..."


In [48]:
# open ai gpt api stuff
import openai
OPENAIKEY="your api key"
OPENAI_ORGANIZATION="org here"
openai.api_key = OPENAIKEY
openai.organization = OPENAI_ORGANIZATION

In [18]:
# model_name = "gpt-4"
# temperature = 0.0
# max_tokens = 1000
stories = list(stories_df.head(1)['story'])
for story in stories:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        # temperature=0.7,
        messages=[
            {"role": "system", "content": "You are a story summarizer. You output a summary of the given story combining Plot and Character summaries and developemnts."},
            {"role": "user", "content": story},
        ]
    )
    print(response['choices'][0]['message']['content'])

Fiona and Grant are a couple who have been married for almost fifty years. Fiona begins showing signs of dementia and starts forgetting things and getting confused about simple tasks. Grant takes her to Meadowlake, a care facility for the elderly. Fiona befriends a man named Aubrey, who is also staying at Meadowlake. Eventually, Fiona's condition worsens and she forgets Grant, mistaking him for a visitor. Grant tries to reconnect with her, but Fiona is focused on her relationship with Aubrey. Grant visits Aubrey's wife, Marian, and asks if Aubrey can visit Fiona, but Marian refuses. Grant realizes that he may not be able to help Fiona and begins to question their relationship. He receives a call from Marian inviting him to a dance for singles, and he contemplates the possibilities of starting a relationship with her. However, when he returns to Meadowlake, he finds Fiona sitting by the window and they share a tender moment, reminding Grant that he still cares deeply for his wife and vo

In [72]:
# prompts to test to get the best summary
story = stories_df.loc[stories_df['Title'] == "My Cheesecake-Shaped Poverty"]['story'].item()

def get_summary(story, model="gpt-3.5-turbo-16k"):
    response = openai.ChatCompletion.create(
            model=model,
            # model = "gpt-4", #gpt-4-32k
            temperature=1.4,
            messages=[
                {"role": "system", "content": "You provide a detailed summary of the given story. Focus on the plot line and character development."},
                {"role": "user", "content": story},
            ]
        )
    return response['choices'][0]['message']['content']
# print(response['choices'][0]['message']['content'])

In [73]:
# append summary as a new column
stories_df = stories_df.assign(summary_gpt_3_5_t_14=stories_df['story'].progress_apply(get_summary))

100%|██████████████████████████████████████████████████████████████████████████████████| 85/85 [10:48<00:00,  7.63s/it]


In [74]:
stories_df

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t_14
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant meet in university and quickly...,Fiona is living in a big disorderly house with...
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...","In this story, the gunman finds himself alone ...","In this story, the protagonist, referred to as..."
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing the bald...","In this story, three men, Tommy, Ernie, and Lu..."
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The story is about the narrator's childhood me...
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, the husband has developed a hab...","In this short story, the writer depicts a dete..."
...,...,...,...,...,...
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...,"Mrs. Packletide, driven by her dislike for Loo...",Mrs. Packletide decides that she wants to shoo...
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...,Aurelio Escovar is a dentist who operates with...,The story is set on a warm and rainless Monday...
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji...","In this story, a man named Jimmy Valentine is ...","Jimmy Valentine, a skilled safe-cracker, is un..."
83,War of the Clowns,Mia Couto,One time two clowns set themselves to arguing....,"In this story, two clowns engage in an argumen...","In the story, two clowns engage in a playful a..."


In [2]:
stories_df = pd.read_csv('stories_with_summary.csv')

In [3]:
stories_df = stories_df.assign(sid = stories_df.apply(lambda x: "s"+str(x.name), axis=1))

In [4]:
stories_df

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t_14,sid
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant meet in university and quickly...,Fiona is living in a big disorderly house with...,s0
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...","In this story, the gunman finds himself alone ...","In this story, the protagonist, referred to as...",s1
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing the bald...","In this story, three men, Tommy, Ernie, and Lu...",s2
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The story is about the narrator's childhood me...,s3
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, the husband has developed a hab...","In this short story, the writer depicts a dete...",s4
...,...,...,...,...,...,...
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...,"Mrs. Packletide, driven by her dislike for Loo...",Mrs. Packletide decides that she wants to shoo...,s80
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...,Aurelio Escovar is a dentist who operates with...,The story is set on a warm and rainless Monday...,s81
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji...","In this story, a man named Jimmy Valentine is ...","Jimmy Valentine, a skilled safe-cracker, is un...",s82
83,War of the Clowns,Mia Couto,One time two clowns set themselves to arguing....,"In this story, two clowns engage in an argumen...","In the story, two clowns engage in a playful a...",s83


In [5]:
stories_df.to_csv("stories_with_summary.csv", index=False)